In [2]:
import polars as pl
import sys
import os
import re  # Import the re module for regular expressions
sys.path.append("/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Fantasy_Football")

In [4]:
path = '/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Bronze/player_stats/receiving'
df_receiving = pl.read_delta(path)
display(df_receiving)

Player,Rec,Yds,TD,20+,40+,LNG,Rec 1st,1st%,Rec FUM,Rec YAC/R,Tgts,Season
str,str,str,str,str,str,str,str,str,str,str,str,i32
"""Michael Thomas""","""125""","""1405""","""9""","""17""","""2""","""72""","""75""","""60""","""2""","""509""","""147""",2018
"""Zach Ertz""","""116""","""1163""","""8""","""13""","""0""","""34""","""66""","""56.9""","""1""","""371""","""156""",2018
"""DeAndre Hopkins""","""115""","""1572""","""11""","""23""","""4""","""49""","""81""","""70.4""","""2""","""402""","""163""",2018
"""Julio Jones""","""113""","""1677""","""8""","""24""","""2""","""58""","""80""","""70.8""","""2""","""470""","""170""",2018
"""Adam Thielen""","""113""","""1373""","""9""","""17""","""3""","""68""","""74""","""65.5""","""1""","""435""","""153""",2018
…,…,…,…,…,…,…,…,…,…,…,…,…
"""Brevin Jordan""","""20""","""178""","""3""","""2""","""0""","""27""","""10""","""50""","""0""","""94""","""28""",2021
"""Isaiah McKenzie""","""20""","""178""","""1""","""1""","""0""","""28""","""13""","""65""","""0""","""81""","""26""",2021
"""Donald Parham""","""20""","""190""","""3""","""1""","""0""","""22""","""13""","""65""","""0""","""138""","""27""",2021


In [6]:
def remove_special_characters(df: pl.DataFrame, column_name: str) -> pl.DataFrame:
    """
    Remove special characters from a specified column in a Polars DataFrame.

    Args:
        df (pl.DataFrame): The input DataFrame.
        column_name (str): The name of the column to modify.

    Returns:
        pl.DataFrame: The modified DataFrame with special characters removed from the specified column.
    """
    # Regular expression pattern to match special characters
    pattern = r"[^\w\s]"
    
    # Remove special characters from the specified column
    df = df.with_columns(
        pl.col(column_name).str.replace_all(pattern, "")
    )
    
    return df

df_player = df.select("Player")
df_player = remove_special_characters(df_player, "Player")

# Cleaning Player data for receivers

In [7]:
df_player_unique = df_player.unique().sort('Player', descending=False)
df_player_unique = df_player_unique.with_row_index(name="id",offset=1)
display(df_player_unique)
#display(df_player_unique.filter(pl.col("Player")== "Antonio Brown"))


id,Player
u32,str
1,"""AB Brown"""
2,"""AJ Brown"""
3,"""AJ Derby"""
4,"""AJ Dillon"""
5,"""AJ Green"""
…,…
5680,"""Zeke Turner"""
5681,"""Zeron Flemister"""
5682,"""Zola Davis"""


In [10]:
df_player_unique = df_player_unique.with_columns([
    pl.col("Player").alias("player_name"),
    pl.col("Player").str.extract(r"(\S+)", 1).alias("first_name"),
    pl.col("Player").str.extract(r"(\S+)$", 0).alias("last_name"),
    pl.lit("receiving").alias("category"),
    pl.lit("offense").alias("position")
])
dim_palyer = df_player_unique.select(["id","player_name",'first_name','last_name', 'position','category'])
print(dim_palyer)

shape: (5_684, 6)
┌──────┬─────────────────┬────────────┬───────────┬──────────┬───────────┐
│ id   ┆ player_name     ┆ first_name ┆ last_name ┆ position ┆ category  │
│ ---  ┆ ---             ┆ ---        ┆ ---       ┆ ---      ┆ ---       │
│ u32  ┆ str             ┆ str        ┆ str       ┆ str      ┆ str       │
╞══════╪═════════════════╪════════════╪═══════════╪══════════╪═══════════╡
│ 1    ┆ AB Brown        ┆ AB         ┆ Brown     ┆ offense  ┆ receiving │
│ 2    ┆ AJ Brown        ┆ AJ         ┆ Brown     ┆ offense  ┆ receiving │
│ 3    ┆ AJ Derby        ┆ AJ         ┆ Derby     ┆ offense  ┆ receiving │
│ 4    ┆ AJ Dillon       ┆ AJ         ┆ Dillon    ┆ offense  ┆ receiving │
│ 5    ┆ AJ Green        ┆ AJ         ┆ Green     ┆ offense  ┆ receiving │
│ …    ┆ …               ┆ …          ┆ …         ┆ …        ┆ …         │
│ 5680 ┆ Zeke Turner     ┆ Zeke       ┆ Turner    ┆ offense  ┆ receiving │
│ 5681 ┆ Zeron Flemister ┆ Zeron      ┆ Flemister ┆ offense  ┆ receiving │
│ 5682 

In [15]:
path = "/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Silver/dim_player"
dim_palyer.write_delta(path,mode="overwrite")

# Cleaning player data for rushing

In [19]:
path = '/Users/treveralexander/Library/CloudStorage/OneDrive-EY/Personal/DE_Project/Storage/Bronze/player_stats/rushing'
df_rushing = pl.read_delta(path)
display(df_rushing)

Player,Rush Yds,Att,TD,20+,40+,Lng,Rush 1st,Rush 1st%,Rush FUM,Season
str,str,str,str,str,str,str,str,str,str,i32
"""Floyd Little""","""1133""","""284""","""6""","""0""","""0""","""40""","""0""","""0""","""0""",1971
"""John Brockington""","""1105""","""216""","""4""","""0""","""0""","""52""","""0""","""0""","""0""",1971
"""Larry Csonka""","""1051""","""195""","""7""","""0""","""0""","""28""","""0""","""0""","""0""",1971
"""Steve Owens""","""1035""","""246""","""8""","""0""","""0""","""23""","""0""","""0""","""0""",1971
"""Willie Ellison""","""1000""","""211""","""4""","""0""","""0""","""80""","""0""","""0""","""0""",1971
…,…,…,…,…,…,…,…,…,…,…
"""Philip Rivers""","""-8""","""18""","""0""","""0""","""0""","""3""","""0""","""0""","""0""",2020
"""Tim Boyle""","""-9""","""13""","""0""","""0""","""0""","""0""","""0""","""0""","""0""",2020
"""Adrian Killins Jr.""","""-12""","""1""","""0""","""0""","""0""","""-12""","""0""","""0""","""0""",2020


In [ ]:
df_rushing_unique = df_rushing.unique().sort('Player', descending=False)
df_rushing_unique = df_player_unique.with_row_index(name="id",offset=1)
display(df_player_unique)
#display(df_player_unique.filter(pl.col("Player")== "Antonio Brown"))

In [ ]:
df_player_unique = df_player_unique.with_columns([
    pl.col("Player").alias("player_name"),
    pl.col("Player").str.extract(r"(\S+)", 1).alias("first_name"),
    pl.col("Player").str.extract(r"(\S+)$", 0).alias("last_name"),
    pl.lit("rushing").alias("category"),
    pl.lit("offense").alias("position")
])
dim_palyer = df_player_unique.select(["id","player_name",'first_name','last_name', 'position','category'])
print(dim_palyer)